In [1]:
#| default_exp uwb

# uwb

In [2]:
#| hide

from nbdev.showdoc import *

## Function define part

In [3]:
#| export

from typing import List
import yaml
import pypozyx
from pypozyx import PozyxSerial
from pypozyx import NetworkID
from pypozyx import Coordinates, DeviceCoordinates
from pypozyx import DeviceRange
from pypozyx import PozyxConstants
from pypozyx.core import PozyxException

In [4]:
#| export

class UWB:
    """
    The main UWB class that handling Pozyx API
    """

    def __init__(self, port: str = None) -> None:
        # public attr
        self.port: str = port
        self.network_id: int = None
        self.pose: Coordinates = None
        self.height: float = 0.0

        # private attr
        self._port_list: List[str] = []
        self._env_config: dict = None
        self._status: int = PozyxConstants.STATUS_SUCCESS

        self._pozyx_handler: PozyxSerial = None


In [5]:
#| export

class UWB(UWB):
    # port
    @property
    def port(self) -> str:
        """A getter method of port string

        The tty device path of Pozyx is like
        `/dev/ttyUAB0` or `/dev/ttyUAB0`

        Returns:
            str: A String of The tty device of Pozyx
        """
        return self._port

    @port.setter
    def port(self, port: str = "") -> None:
        """A setter method of port string

        Args:
            port (str, optional): A String of The tty device of Pozyx. Defaults to "".
        """
        self._port = port


In [6]:
#| export

class UWB(UWB):
    # network_id
    @property
    def network_id(self) -> int:
        """A getter method of network id

        The network id of Pozyx is in hex and like `0x6A27`,
        but showing in dec is like `27175`.
        If want get network id in readable, see network_id_str

        Returns:
            int: A decimal id number of The Pozyx
        """
        return self._network_id.id
    
    @property
    def network_id_str(self) -> str:
        """A getter method of network id string

        Convert network id to string to show in readable.

        Returns:
            str: A string of id number in hexadecimal of The Pozyx
        """
        return str(self._network_id)

    @network_id.setter
    def network_id(self, value: int = None) -> None:
        """A setter method of port string

        Args:
            value (int, optional): A integer id number in hexadecimal or decimal of The Pozyx. Defaults to None.
        """
        if value is None:
            self._network_id = NetworkID()
        else:
            self._network_id = NetworkID(value)


In [7]:
#| export

class UWB(UWB):
    # pose
    @property
    def pose(self) -> List[float]:
        """A getter method of UWB pose

        Returns:
            list[float]: (pose.x, pose.y, pose.z)
        """
        return (self._pose.x, self._pose.y, self._pose.z)

    @pose.setter
    def pose(self, value: List[float] = None) -> None:
        """A setter method of UWB pose

        Args:
            value (List[float], optional): (pose.x, pose.y, pose.z) Defaults to None.
        """
        if value is None:
            self._pose = Coordinates()
        else:
            self._pose.x = value[0]
            self._pose.y = value[1]
            self._pose.z = value[2]


In [8]:
#| export

class UWB(UWB):
    # height
    @property
    def height(self) -> float:
        """A getter method of UWB pose height

        Returns:
            float: The default height for 2.5D localization.
        """
        return self._height

    @height.setter
    def height(self, value: float = 0) -> float:
        """A setter method of UWB pose height

        Args:
            value (int, optional): The default height for 2.5D localization.. Defaults to 0.
        """
        self._height = value


In [9]:
#| export

class UWB(UWB):
    # env_config
    @property
    def env_config(self) -> dict:
        """A getter method of environment config

        Returns:
            dict: The environment config in dict format.
        """
        return self._env_config


In [10]:
#| export


class UWB(UWB):
    # port_lost
    def port_list(self) -> List[str]:
        """A getter method of port list.

        Returns:
            List[str]: The list contains UWB port device path like `/dev/ttyACM0`.
        """
        return self._port_list


In [11]:
#| export


class UWB(UWB):
    # status
    @property
    def status(self) -> int:
        """A getter method of UWB status.

        Returns:
            int: The status got from Pozyx. 0 is success.
        """
        return self._status


In [12]:
#| export

class UWB(UWB):
    def load_env_config(self, config_file_path: str) -> bool:
        """Load UWB anchors' environment config.

        Args:
            config_file_path (str): The environment config file path.

        Returns:
            bool: True for success, False for failure.
        """
        with open(config_file_path, "r") as config_file:
            try:
                self._env_config = yaml.safe_load(config_file)
            except yaml.YAMLError as ex:
                print(ex)
                return False
        return True


In [13]:
#| export

class UWB(UWB):
    def scan_port(self) -> None:
        """Scan all port connecting to host. Store port device path in port list.
        """
        self._port_list = pypozyx.get_pozyx_ports()


In [14]:
#| export

class UWB(UWB):
    def connect(self) -> bool:
        """Try to connect pozyx device.

        Returns:
            bool: Pozyx status
        """
        self._status = PozyxConstants.STATUS_SUCCESS
        if self.port is None:
            self.scan_port()
            if len(self._port_list) == 1:
                self.port = self._port_list[0]
                self._pozyx_handler = PozyxSerial(self.port)
                self._status &= self._pozyx_handler.getNetworkId(self._network_id)
            elif len(self._port_list) == 0:
                return False
            else:
                return False
        else:
            try:
                self._pozyx_handler = PozyxSerial(self.port)
                self._status &= self._pozyx_handler.getNetworkId(self._network_id)
                return True
            except PozyxException as ex:
                print(ex)
                return False


In [15]:
#| export

class UWB(UWB):
    def write_env_config(self) -> bool:
        """Write environment anchor location into Pozyx UWB device.

        Returns:
            bool: Pozyx status
        """
        self._status = PozyxConstants.STATUS_SUCCESS
        ANCHOR_FLAG = 1
        self._status &= self._pozyx_handler.clearDevices()
        for anchor_name, config in self.env_config.items():
            coordinate = Coordinates(config["x"], config["y"], config["z"])
            device_coordinate = DeviceCoordinates(config["id"], ANCHOR_FLAG, coordinate)
            self._status &= self._pozyx_handler.addDevice(device_coordinate)
        if len(self.env_config) > 4:
            self._status &= self._pozyx_handler.setSelectionOfAnchorsAutomatic(len(self.env_config))
        return self._status


In [16]:
#| export

class UWB(UWB):
    def localize_2_5D(self) -> bool:
        """Localize method in 2.5D. Need to know height.

        Returns:
            bool: Pozyx status
        """
        self._status &= self._pozyx_handler.doPositioning(
            self._pose,
            PozyxConstants.DIMENSION_2_5D,
            self._height,
            PozyxConstants.POSITIONING_ALGORITHM_UWB_ONLY,
        )
        return self._status


In [17]:
#| export

class UWB(UWB):
    def localize_3D(self)->bool:
        """Localize method in 3D. The height will be determined by Pozyx UWB device.

        Returns:
            bool: Pozyx status
        """
        self._status &= self._pozyx_handler.doPositioning(
            self._pose,
            PozyxConstants.DIMENSION_3D,
            self._height,
            PozyxConstants.POSITIONING_ALGORITHM_UWB_ONLY,
        )
        return self._status


In [18]:
#| export

class UWB(UWB):
    def range_from(self, dest_id) -> float:
        """Range method from this Pozyx UWB device to the destination Pozyx UWB device.

        Args:
            dest_id (_type_): The target Pozyx UWB device id want to be ranged.

        Returns:
            float: The range from this Pozyx UWB device to the destination Pozyx UWB device.
        """
        ranges = DeviceRange()
        self._pozyx_handler.doRanging(dest_id, ranges)
        return ranges


## Testing example


In [19]:
#| hide

In [20]:
!cat data/duckiepond-devices/uwb_ee6f.yaml

anchor1:
  id: 0x6a42
  x: 0
  y: 2570
  z: 1000

anchor2:
  id: 0x6a60
  x: 0
  y: 0
  z: 1000
  
anchor3:
  id: 0x6e5d
  x: 18000
  y: 0
  z: 1000

anchor4:
  id: 0x6713
  x: 20800
  y: 0
  z: 1000

anchor5:
  id: 0x6a2f
  x: 20800
  y: 2570
  z: 1000

# anchor6:
#   id: 0x6a41
#   x: 22000
#   y: 17430
#   z: 1000

# anchor7:
#   id: 0x670c
#   x: 0
#   y: -1350
#   z: 1000

# anchor8:
#   id: 0x6a78
#   x: 0
#   y: -1350
#   z: 1000


In [21]:
uwb = UWB()
# TODO: Add scan port method
uwb.connect()
uwb.height = 1000
uwb.load_env_config("data/duckiepond-devices/uwb_ee6f.yaml")
uwb.write_env_config()

1

In [22]:
str(uwb.network_id_str)

'0x6A7E'

In [23]:
uwb.env_config

{'anchor1': {'id': 27202, 'x': 0, 'y': 2570, 'z': 1000},
 'anchor2': {'id': 27232, 'x': 0, 'y': 0, 'z': 1000},
 'anchor3': {'id': 28253, 'x': 18000, 'y': 0, 'z': 1000},
 'anchor4': {'id': 26387, 'x': 20800, 'y': 0, 'z': 1000},
 'anchor5': {'id': 27183, 'x': 20800, 'y': 2570, 'z': 1000}}

In [24]:
import time

ranges = [0.0, 0.0, 0.0, 0.0]
for i in range(50):
    ranges[0] = uwb.range_from(0x6a42).distance
    ranges[1] = uwb.range_from(0x6a60).distance
    ranges[2] = uwb.range_from(0x6e5d).distance
    ranges[3] = uwb.range_from(0x6713).distance
    print('[', end='')
    for r in ranges:
        print(r, end=", ")
    print(']', end='\r+')
#     print(f"0x6a42: {uwb.range_from(0x6a42)}")
#     print(f"0x6a60: {uwb.range_from(0x6a60)}")
#     print(f"0x6e5d: {uwb.range_from(0x6e5d)}")
#     print(f"0x6713: {uwb.range_from(0x6713)}")
    time.sleep(0.05)


In [25]:
for _ in range(2000):
    uwb.localize_2_5D()
    print(uwb.pose[0], end=", ")
    print(uwb.pose[1], end=", ")
    print(uwb.pose[2])
    time.sleep(0.05)


1856.0, 333.0, 1000.0
1931.0, 435.0, 1000.0
2073.0, 621.0, 1000.0
2224.0, 872.0, 1000.0
2148.0, 699.0, 1000.0
2124.0, 660.0, 1000.0
1916.0, 358.0, 1000.0
1961.0, 433.0, 1000.0
2326.0, 1038.0, 1000.0
1974.0, 470.0, 1000.0
1969.0, 426.0, 1000.0
2054.0, 554.0, 1000.0
2064.0, 505.0, 1000.0
2051.0, 492.0, 1000.0
2182.0, 612.0, 1000.0
2150.0, 570.0, 1000.0
2178.0, 544.0, 1000.0
2197.0, 625.0, 1000.0
2215.0, 645.0, 1000.0
2181.0, 638.0, 1000.0
2159.0, 574.0, 1000.0
2191.0, 595.0, 1000.0
2199.0, 622.0, 1000.0
2136.0, 566.0, 1000.0
2347.0, 857.0, 1000.0
2182.0, 662.0, 1000.0
2174.0, 761.0, 1000.0
2153.0, 546.0, 1000.0
1968.0, 465.0, 1000.0
1996.0, 468.0, 1000.0
2100.0, 526.0, 1000.0
1980.0, 481.0, 1000.0
2177.0, 624.0, 1000.0
2165.0, 600.0, 1000.0
2430.0, 1093.0, 1000.0
2057.0, 539.0, 1000.0
2234.0, 677.0, 1000.0
2179.0, 601.0, 1000.0
2340.0, 989.0, 1000.0
2026.0, 502.0, 1000.0
2069.0, 509.0, 1000.0
1917.0, 384.0, 1000.0
1960.0, 423.0, 1000.0
2535.0, 1695.0, 1000.0
2469.0, 1898.0, 1000.0
1945.0

-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 72

-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 72

-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 72

-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 72

-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 720.0, 1000.0
-361.0, 72